In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time


In [2]:
data = pd.read_csv('./일반음식점_송파.csv',encoding='cp949')
data

,업종명,업소명,소재지(도로명),소재지전화
0,일반음식점,샹하이,"서울특별시 송파구 석촌호수로 82, 지상1층 (잠실동)",02-416-2283
1,일반음식점,영월식당,"서울특별시 송파구 마천로51가길 2, 지상1층 (마천동)",02-400-2665
2,일반음식점,영미식당,"서울특별시 송파구 충민로 81-14, 1,2,3층 (문정동)",02-402-1007
3,일반음식점,국일반점,서울특별시 송파구 마천로 345-1 (마천동),02-406-1798
4,일반음식점,부산회집,"서울특별시 송파구 올림픽로35길 94 (신천동,장미C상가 2,3,4호)",02-418-4624
...,...,...,...,...
7045,일반음식점,BK wing(비케이 윙),"서울특별시 송파구 양재대로64길 28, 지층 A-1호 (가락동)",NaN
7046,일반음식점,당신은 지금 치킨이 땡긴다 석촌점,"서울특별시 송파구 석촌호수로 210, 지하1층 11호 (석촌동, 석촌호수효성해링턴타워)",NaN
7047,일반음식점,선한사람들,"서울특별시 송파구 법원로4길 6, 문정아이파크 108호 중 6호 (문정동)",NaN
7048,일반음식점,중경식객 송파점,"서울특별시 송파구 동남로18길 31, 1층 (가락동)",NaN


In [3]:
restaurant_dict = {}
for i in tqdm(data['업소명']):    
    #print(i)
    target = '송파구+' + i
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query={}'.format(target)
    req = requests.get(url)

    html = req.text
    soup = BeautifulSoup(html,'html.parser')
    #1) 검색시 여러개 나오는 경우
    b =  soup.find_all(class_='VLTHu')
    if len(b) >0:
        for i in b:
            #print(i.find(class_= "C6RjW").find('span').text) # 해당 이름
            if str(i.find(class_="hClKF")).find("서울 송파구") != -1:
                tmp = i.get('data-loc_plc-doc-id')
                #print(i.get('data-loc_plc-doc-id'))
                restaurant_dict['{}'.format(tmp)] = i.find(class_= "C6RjW").find('span').text
    else:
        #2. 검색해서 하나만 나오는 경우
        b =  soup.find(class_='zD5Nm')
        if b != None:# 검색시 안나오는 곳일 경우에는 find를 사용시 에러나서 방어함
            result = re.findall(r'/[0-9]+', str(b.find(class_='YouOG').a))
            #print(result[0][1:]) #해당 타겟 id
            restaurant_dict['{}'.format(result[0][1:])] = b.find('span').text
    #print(restaurant_dict)
    time.sleep(0.5)    



  1%|          | 52/7050 [00:54<2:03:14,  1.06s/it]


KeyboardInterrupt: 

In [ ]:
test_df = pd.DataFrame.from_dict(restaurant_dict, orient='index').reset_index()
test_df.to_csv('./crwaling.csv', index=False)


In [ ]:
test_df

In [ ]:
#process.py
#restaurant_dict = {}
#url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query={}'.format('송파구+상하이')
#req = requests.get(url)

#html = req.text
#soup = BeautifulSoup(html,'html.parser')


In [ ]:
#1) 검색시 여러개 나오는 경우
#b =  soup.find_all(class_='VLTHu')
#print(len(b))
#for i in b:
    #print(i.find(class_= "C6RjW").find('span').text) # 해당 이름
    #if str(i.find(class_="hClKF")).find("서울 송파구") != -1:
        #tmp = i.get('data-loc_plc-doc-id')
        #print(i.get('data-loc_plc-doc-id'))
        #restaurant_dict['{}'.format(tmp)] = i.find(class_= "C6RjW").find('span').text

6


In [ ]:
#2. 검색해서 하나만 나오는 경우
#b =  soup.find(class_='zD5Nm')
#print(b.find('span').text)
#print(b.attrs)
#print(b.find(class_='YouOG').a)
#print(b)
#print(b.find_all('em'))
#print(len(b.find_all('em')))
#result = re.findall(r'/[0-9]+', str(b.find(class_='YouOG').a))
#print(result[0][1:]) #해당 타겟 id
#restaurant_dict['{}'.format(result[0][1:])] = b.find('span').text

In [ ]:
#test_df = pd.DataFrame.from_dict(restaurant_dict, orient='index').reset_index()
#test_df
#test_df.to_csv('./crwaling.csv', index=False)